
### 1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>. You have to use data.csv file for this assignment
### 2. Code the model to classify data like below image. You can use any number of units in your Dense layers.

<img src='https://i.imgur.com/33ptOFy.png'>



# <font color='red'> <b>3. Writing Callbacks </b> </font>
## You have to implement the following callbacks
-  Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

- Save your model at every epoch if your validation accuracy is improved from previous epoch. 

- You have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
- If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

- You have to stop the training if your validation accuracy is not increased in last 2 epochs.

- Use tensorboard for every model and analyse your scalar plots and histograms. (you need to upload the screenshots and write the observations for each model for evaluation)



<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

##Writing Callbacks

##Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.Do not use tf.keras.metrics for calculating AUC and F1 score.

In [1]:
from google.colab import files
files=files.upload()

Saving data.csv to data.csv


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler


In [5]:
data=pd.read_csv("data.csv")
data.head()

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [7]:
X = data.drop(['label'], axis=1).values
Y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, stratify=Y) 
#X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(13400, 2)
(13400,)
(6600, 2)
(6600,)


In [12]:
class Metrics(tf.keras.callbacks.Callback):

   def on_train_begin(self, logs={}):
     self.val_f1s = []
     
   def on_epoch_end(self, epoch, logs={}):
     #val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
     val_predict = (np.asarray(self.model.predict(X_test))).round()
     #val_targ = self.model.validation_data[1]
     _val_f1 = f1_score(y_test, val_predict,average='micro') 
     self.val_f1s.append(_val_f1)
     #print(" value f1 ",_val_f1)
     print("  f1_score: "+"{:.4f}".format(_val_f1)); 
     return

history_own=Metrics()
#print(history_own.val_f1s)

In [13]:
#Input layer
input_layer = Input(shape=(2,))

#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)

#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)

#Model Creation
model = Model(inputs=input_layer,outputs=output)

#Now Callbacks:
#history_own = LossHistory()
history_own  = Metrics()

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])

model.fit(X_train,y_train,epochs=5, validation_data=(X_test,y_test), batch_size=20, callbacks=[history_own])

Epoch 1/5
670/670 [==============================] - 2s 3ms/step - loss: 0.7014 - auc: 0.5003 - val_loss: 0.6958 - val_auc: 0.4914
Epoch 2/5
670/670 [==============================] - 2s 2ms/step - loss: 0.6934 - auc: 0.5134 - val_loss: 0.6935 - val_auc: 0.4972
Epoch 3/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6928 - auc: 0.5143 - val_loss: 0.6929 - val_auc: 0.5116
Epoch 4/5
670/670 [==============================] - 3s 4ms/step - loss: 0.6925 - auc: 0.5254 - val_loss: 0.6927 - val_auc: 0.5230
Epoch 5/5
670/670 [==============================] - 2s 4ms/step - loss: 0.6923 - auc: 0.5309 - val_loss: 0.6924 - val_auc: 0.5329


In [14]:
history_own.val_f1s

[0.5077272727272727,
 0.4860606060606061,
 0.5013636363636363,
 0.5075757575757576,
 0.5209090909090909]

In [15]:
#If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training.
class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True


In [16]:
terminate= TerminateNaN()
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[terminate])

Epoch 1/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6921 - auc: 0.5437 - val_loss: 0.6922 - val_auc: 0.5359
Epoch 2/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6918 - auc: 0.5506 - val_loss: 0.6921 - val_auc: 0.5419
Epoch 3/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6915 - auc: 0.5587 - val_loss: 0.6919 - val_auc: 0.5388
Epoch 4/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6912 - auc: 0.5590 - val_loss: 0.6915 - val_auc: 0.5490
Epoch 5/5
670/670 [==============================] - 3s 4ms/step - loss: 0.6909 - auc: 0.5636 - val_loss: 0.6910 - val_auc: 0.5618


In [21]:
#Save your model at every epoch if your validation accuracy is improved from previous epoch.

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

#Callbacks
#file path, it saves the model in the 'model_save' folder and we are naming model with epoch number 
#and val auc to differtiate with other models
#you have to create model_save folder before running the code.

filepath="D:\Applied AI Course\Assignments\20. Assignment- Working with Callbacks\model_save5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])

model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[checkpoint])

Epoch 1/5
668/670 [============================>.] - ETA: 0s - loss: 0.7350 - auc: 0.4993
Epoch 00001: val_auc improved from -inf to 0.49661, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save5
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save5/assets
670/670 [==============================] - 3s 3ms/step - loss: 0.7349 - auc: 0.4995 - val_loss: 0.6966 - val_auc: 0.4966
Epoch 2/5
652/670 [============================>.] - ETA: 0s - loss: 0.6935 - auc: 0.5020
Epoch 00002: val_auc improved from 0.49661 to 0.53466, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save5
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save5/assets
670/670 [==============================] - 2s 3ms/step - loss: 0.6935 - auc: 0.5024 - val_loss: 0.6925 - val_auc: 0.5347
Epoch 3/5
647/670 [========

In [25]:
#You have to stop the training if your validation accuracy is not increased in last 2 epochs.

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')


optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[earlystop])

Epoch 1/5
670/670 [==============================] - 2s 2ms/step - loss: 0.7426 - auc: 0.4988 - val_loss: 0.7068 - val_auc: 0.4982
Epoch 2/5
670/670 [==============================] - 2s 2ms/step - loss: 0.6992 - auc: 0.5014 - val_loss: 0.6962 - val_auc: 0.4979
Epoch 3/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6941 - auc: 0.5083 - val_loss: 0.6941 - val_auc: 0.4978
Epoch 00003: early stopping


In [27]:
#You have to decay learning rate on the basis of following conditions:

#Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the learning rate by 10%. 
#Cond2. For every 3rd epoch, decay your learning rate by 5%.

def changeLearningRate(epoch):
    initial_learningrate=0.01
    if epoch % 3 ==0:
      changed = initial_learningrate*(1-0.05)**epoch
    else:
      changed = initial_learningrate*(1-0.1)**epoch
    return changed

changed_lr = []
for i in range(1,10):
  changed_lr.append(changeLearningRate(i))

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="model_save/weights-{epoch:02d}-{val_auc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/20
668/670 [============================>.] - ETA: 0s - loss: 0.7179 - auc: 0.5021  f1_score: 0.5023

Epoch 00001: val_auc improved from -inf to 0.50129, saving model to model_save/weights-01-0.5013.hdf5
670/670 [==============================] - 2s 3ms/step - loss: 0.7179 - auc: 0.5019 - val_loss: 0.6962 - val_auc: 0.5013 - lr: 0.0100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009000000000000001.
Epoch 2/20
664/670 [============================>.] - ETA: 0s - loss: 0.6930 - auc: 0.5212  f1_score: 0.5136

Epoch 00002: val_auc improved from 0.50129 to 0.52587, saving model to model_save/weights-02-0.5259.hdf5
670/670 [==============================] - 2s 3ms/step - loss: 0.6929 - auc: 0.5218 - val_loss: 0.6928 - val_auc: 0.5259 - lr: 0.0090

Epoch 00003: LearningRateScheduler setting learning rate to 0.008100000000000001.
Epoch 3/20
653/670 [============================>.] - ETA: 0s - loss: 0.

***Model 1 Observations:***

1.   Epoch No. 6 given Maximum F1 Score: 0.6138 & val_auc= 0.6222
2.   Initially, val_auc increases gradually, after Epoch number 5 val_auc is not increasing
3.   As Epoch number increases, val_loss decreases











<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [30]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="D:\Applied AI Course\Assignments\20. Assignment- Working with Callbacks\model_save"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/20
664/670 [============================>.] - ETA: 0s - loss: 0.7053 - auc: 0.4446  f1_score: 0.4689

Epoch 00001: val_auc improved from -inf to 0.39784, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save/assets
670/670 [==============================] - 3s 4ms/step - loss: 0.7053 - auc: 0.4441 - val_loss: 0.6961 - val_auc: 0.3978 - lr: 0.0100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009000000000000001.
Epoch 2/20
659/670 [============================>.] - ETA: 0s - loss: 0.6941 - auc: 0.4696  f1_score: 0.4802

Epoch 00002: val_auc improved from 0.39784 to 0.51278, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. As

***Model 2 Observations:***

1.   Epoch No. 5 given Maximum F1 Score: 0.5215 & val_auc= 0.5290
2.   Initially, val_auc increases gradually, after Epoch number 3 val_auc is not increasing
3.   As Epoch number increases, val_loss decreases







<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>

In [31]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="D:\Applied AI Course\Assignments\20. Assignment- Working with Callbacks\model_save"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/20
660/670 [============================>.] - ETA: 0s - loss: 0.6996 - auc: 0.5056  f1_score: 0.5385

Epoch 00001: val_auc improved from -inf to 0.57546, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save/assets
670/670 [==============================] - 3s 4ms/step - loss: 0.6995 - auc: 0.5063 - val_loss: 0.6869 - val_auc: 0.5755 - lr: 0.0100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009000000000000001.
Epoch 2/20
669/670 [============================>.] - ETA: 0s - loss: 0.6846 - auc: 0.5844  f1_score: 0.5633

Epoch 00002: val_auc improved from 0.57546 to 0.60760, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. As

***Model 3 Observations:***

1.   Epoch No. 20 given Maximum F1 Score: 0.6244 & val_auc= 0.6645
2.   Initially, as epoch number increases, val_auc is also increases
3.   As Epoch number increases, val_loss decreases





<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [32]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='selu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="D:\Applied AI Course\Assignments\20. Assignment- Working with Callbacks\model_save"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/20
636/670 [===========================>..] - ETA: 0s - loss: 0.6953 - auc: 0.5119  f1_score: 0.5620

Epoch 00001: val_auc improved from -inf to 0.57145, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save/assets
670/670 [==============================] - 3s 4ms/step - loss: 0.6951 - auc: 0.5130 - val_loss: 0.6902 - val_auc: 0.5714 - lr: 0.0100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009000000000000001.
Epoch 2/20
649/670 [============================>.] - ETA: 0s - loss: 0.6889 - auc: 0.5763  f1_score: 0.5686

Epoch 00002: val_auc improved from 0.57145 to 0.61152, saving model to D:\Applied AI Course\Assignments. Assignment- Working with Callbacks\model_save
INFO:tensorflow:Assets written to: D:\Applied AI Course\Assignments. As

***Model 4 Observations:***

1.   Epoch No. 4 given Maximum F1 Score: 0.5892 & val_auc= 0.6150
2.   As Epoch number increases, val_loss decreases

